In [3]:
import osmnx as ox # visualisation library for networks
import pickle
from collections import defaultdict
from tqdm import tqdm
from sklearn.neighbors import BallTree

In [4]:
# Load graph
with open('rejsekort_graph_cleaned.gpickle', 'rb') as f:
    G = pickle.load(f)

In [5]:
def tacicab_sphere(latlong1,latlong2):
    '''
    Returns the taxicab distance between two points on a sphere (given lat-long pairs).
    Formula: http://cs.ucmo.edu/~mjms/2005.1/bayar.pdf
    '''
    x1,y1 = latlong1
    x2,y2 = latlong2
    return min(
        geodesic((x1,y1),(x2,y1)).meters + geodesic((x2,y1),(x2,y2)).meters,
        geodesic((x1,y1),(x1,y2)).meters + geodesic((x1,y2),(x2,y2)).meters
    )

def m2s(m):
    return m/1.25 #Walking speed used by Google Maps

# Conversion function for input to BallTree
def convert_latlon_to_balltree_input(lat, lon):
    return np.vstack([np.radians(lon), np.radians(90-np.array(lat))]).T

In [6]:
def add_walk_edges(G):
    '''
    Adds walk edges to network using the BallTree algorithm for fast neighbor detection
    '''
    G = G.copy()
    node_positions = convert_latlon_to_balltree_input([G.nodes[node]["y"] for node in G.nodes()], [G.nodes[node]["x"] for node in G.nodes()])
    balltree = BallTree(node_positions, metric='haversine')

    for idx1, node1 in enumerate(tqdm(G.nodes())):
        node1_position = convert_latlon_to_balltree_input([G.nodes[node1]["y"]], [G.nodes[node1]["x"]])
        
        # Use the BallTree to find all nodes within 1 km
        # Divide by the Earth's radius, which is approximately 6371 kilometres, to convert to radians
        close_nodes = balltree.query_radius(node1_position, r=1.0 / 6371)
        close_nodes = close_nodes[0]  # query_radius returns a list of arrays, we are interested in the first array

        for idx2 in close_nodes:
            node2 = list(G.nodes())[idx2]
            if node1 == node2:
                continue
            for (u,v,edge_type) in G.edges(node1, data="mode"):
                if v == node2 and edge_type in {"transfer","walk"}:
                    break
            else:
                walk_time = m2s(tacicab_sphere((G.nodes[node1]["x"],G.nodes[node1]["y"]),(G.nodes[node2]["x"],G.nodes[node2]["y"])))
                if walk_time <= 30*60 or node2 in G.neighbors(node1): #If less than x time away or transit edge (non-walk, non-transfer neigbor-edges)
                    G.add_edge(node1, node2, routes = {}, trips = {}, mode = 'walk', weight=max(walk_time,180))
    return G

In [41]:
G_walks = add_walk_edges(G)

100%|█████████████████████████████████████████████████████████████████████████████| 38172/38172 [15:40<00:00, 40.61it/s]


In [53]:
# Save new graph to a pickle file
with open('G_walks.pkl', 'wb') as file:
    pickle.dump(G_walks, file)